## TODO:

- [x] Split the dataset into train and test csv files 
- [ ] Implement **WandB** logging
- [ ] Use the train set to create embeddings using the triplet loss and contrastive loss
- [ ] Perform Face Classification
- [ ] Compare results
- [ ] Add to report: 
        * Follow this structure: https://www.diva-portal.org/smash/get/diva2:1327708/FULLTEXT01.pdf
- [ ] Reorganize project 

In [7]:
import numpy as np
import tqdm
import torch
import os
import pandas as pd

from losses.contrastive_loss import ContrastiveLoss, ContrastiveDataset
from losses.triplet_loss import TripletLoss, TripletDataset
from facenet_pytorch import InceptionResnetV1, training
from torch.optim import Adam

from losses.contrastive_loss import ContrastiveLoss, ContrastiveDataset
from torch.utils.data import DataLoader, SequentialSampler

### Setting up WandB

In [2]:
import wandb

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="PRinAI",
    name="100_epochs_ContrastiveLoss",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.001,
    "backbone": "InceptionResnetV1",
    "optimizer": "Adam",
    "batch_size": 24,
    "dataset": "LFW",
    "epochs": 10,
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dodz. Use `wandb login --relogin` to force relogin


### Define some constants

In [2]:
train_csv = 'lfw_csvs/lfw_cropped_train.csv'
test_csv = 'lfw_csvs/lfw_cropped_test.csv'

batch_size = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'
random_seed = 42

## Triplet Loss

### Load the triplets dataset

In [ ]:
from torch.utils.data import DataLoader, SequentialSampler


triplet_dataset = TripletDataset(csv_file='/home/diaa/My_PRinAI/lfw_csvs/lfw_cropped_train.csv')

triplet_loader = DataLoader(
    triplet_dataset,
    num_workers=4,
    pin_memory=True,
    batch_size=batch_size,
    sampler=SequentialSampler(triplet_dataset)
)

### Create triplet Model

In [ ]:
# Create an inception resnet (in train mode):
backbone_cont = InceptionResnetV1(
    classify=False,
    num_classes=len(triplet_dataset.class_to_idx)
    ).to(device)

# Train the model for 5 epochs with triplet loss
optimizer = torch.optim.AdamW(backbone_cont.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [5, 10])

# Define the triplet loss function
triplet_loss = TripletLoss(margin=14).to(device)

loss_fn = triplet_loss
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [ ]:
def train_net_triplet_loss(model, loader, optimizer, scheduler, epochs):
    model.train()
    
    for epoch in tqdm.tqdm(range(epochs)):
        for batch in loader:
            # model.zero_grad()  # what's the difference to optimizer.zero_grad()?
            anchor_embedding = model(batch["anchor"].to(device))
            positive_embedding = model(batch["positive"].to(device))
            negative_embedding = model(batch["negative"].to(device))
            loss = triplet_loss(anchor_embedding,
                                 positive_embedding,
                                 negative_embedding)
            loss.backward()
            optimizer.step()

        scheduler.step()
        wandb.log({"loss": loss}, step=epoch)
        
        # Every 100 epochs save model
        try:
            if epoch % 100 == 0:
                print(f"Epoch {epoch} loss: {loss} ")
                torch.save(model.state_dict(), f"models/100_epochs_triplet_loss_{epoch}.pth")
                print(f"Model saved at epoch {epoch}")
                
        except Exception as e:
            print(f"Error saving model at epoch {epoch}: {e}")
            pass

            
# Creates once at the beginning of training
train_net_triplet_loss(model=backbone_cont,
                       loader=triplet_loader,
                       optimizer=optimizer,
                       scheduler=scheduler,
                       epochs=1000)

## Contrastive Loss

### Load the contrastive dataset

In [11]:
contrastive_dataset = ContrastiveDataset(csv_file='/home/diaa/My_PRinAI/lfw_csvs/lfw_cropped_train.csv')
contrastive_loader = DataLoader(contrastive_dataset, 
                    num_workers=4,
                    pin_memory=True,
                    batch_size=batch_size, 
                    shuffle=True)

### Create contrastive Model

In [6]:
# Create an inception resnet (in train mode):
backbone_cont = InceptionResnetV1(
    classify=False,
    num_classes=len(contrastive_dataset.class_to_idx)
    ).to(device)

# Using Adam optimizer
optimizer = torch.optim.AdamW(backbone_cont.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [5, 10])

contras_loss = ContrastiveLoss().to(device)

metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [7]:
def train_net_contrastive_loss(model, loader, optimizer, scheduler, epochs):
    model.train()
    
    for epoch in tqdm.tqdm(range(epochs)):
        for i, (anchor, target_img, label) in enumerate(loader):
            anchor = anchor.to(device)
            target_img = target_img.to(device)
            
            # Compute embeddings
            embeddings_anchor = model(anchor)
            embeddings_target = model(target_img)
            label = label.to(device)
            loss = contras_loss(embeddings_anchor, embeddings_target, label)

            loss.backward()
            optimizer.step()

        scheduler.step()
        wandb.log({"loss": loss}, step=epoch)
        
        # Every 50 epochs save model
        try:
            if epoch % 50 == 0:
                print(f"Epoch {epoch} loss: {loss} ")
                torch.save(model.state_dict(), f"models/50_epochs_contrastive_loss_{epoch}.pth")
                print(f"Model saved at epoch {epoch}")
                
        except Exception as e:
            print(f"Error saving model at epoch {epoch}: {e}")
            pass

            
# Creates once at the beginning of training
train_net_contrastive_loss(model=backbone_cont,
                            loader=contrastive_loader,
                            optimizer=optimizer,
                            scheduler=scheduler,
                            epochs=200)

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch 0 loss: 1.1064393520355225 


  0%|          | 1/200 [00:46<2:35:00, 46.74s/it]

Model saved at epoch 0


 25%|██▌       | 50/200 [35:48<1:46:44, 42.70s/it]

...

Epoch 100 loss: 0.9342449903488159 


 50%|█████     | 101/200 [1:14:15<1:19:42, 48.31s/it]

Model saved at epoch 100


 53%|█████▎    | 106/200 [1:19:14<1:10:16, 44.85s/it]


KeyboardInterrupt: 

## Compare Loss

### Implement a simple Siamese network for face verification

In [3]:
from torch import nn
import torch.nn.functional as F


class SiameseNetwork(nn.Module):
    def __init__(self, embedding_model):
        super(SiameseNetwork, self).__init__()
        
        self.embedding_model = embedding_model

    def forward(self, input1, input2):
        output1 = self.embedding_model(input1)
        output2 = self.embedding_model(input2)
        
        # Calculate Euclidean distance between the output embeddings
        distance = F.pairwise_distance(output1, output2)
        
        return distance

In [9]:
del backbone_cont
torch.cuda.empty_cache()

In [19]:
def compare_embeddings(embedding_model_path=None):
    # load the trained triplet loss state dict
    embedding_model = InceptionResnetV1(
        classify=False,
        num_classes=len(contrastive_dataset.class_to_idx)
        ).to(device)
    
    loss_name = "backbone model"
    if embedding_model_path:
        embedding_model.load_state_dict(torch.load(embedding_model_path))
        loss_name = embedding_model_path.split("_")[2] + " loss"
        
    # embedding_model.load_state_dict(torch.load("models/100_epochs_triplet_loss_100.pth"))

    # verification_model = SiameseNetwork(triplet_embedding_model).to(device)
    embedding_model.eval()

    # Create the siames model
    verification_model = SiameseNetwork(embedding_model).to(device)

    # Test the verification model
    verification_model.eval()

    # Create a test dataset and loader
    test_dataset = ContrastiveDataset(csv_file='/home/diaa/My_PRinAI/lfw_csvs/lfw_cropped_test.csv')
    test_loader = DataLoader(
        test_dataset,
        num_workers=4,
        pin_memory=True,
        batch_size=1,
        sampler=SequentialSampler(test_dataset)
    )

    distance = []
    results = []
    for i, batch in enumerate(test_loader):
        # Get the anchor and positive images
        anchor = batch[0].to(device)
        target = batch[1].to(device)
        label = batch[2][0]
        
        # Is target positive or negative?
        match = False if label == 0 else True
            
        # Pass the face images through the Siamese network for verification
        distance = verification_model(anchor, target)

        # Define a threshold for face verification
        threshold = 0.5

        # Compare the distance with the threshold
        if distance < threshold:
            pred_match = True
        else:
            pred_match = False
                    
        if match == pred_match:
            results.append(1)
        else:
            results.append(0)
            
    accuracy = sum(results) / len(results)
    print(f"Accuracy on test set with {loss_name}: {accuracy}")
    
    return distance, results, accuracy
        

In [20]:
distance_contrastive, results_contrastive, accuracy_contrastive = compare_embeddings("models/50_epochs_contrastive_loss_100.pth")
distance_triplet, results_triplet, accuracy_triplet = compare_embeddings("models/100_epochs_triplet_loss_100.pth")

Accuracy on test set with contrastive loss: 0.3683409436834094
Accuracy on test set with triplet loss: 0.4094368340943683


In [23]:
# Print the mean distance for each loss function
print(f"Mean distance for triplet loss: {float(distance_triplet.mean())}")
print(f"Mean distance for contrastive loss: {float(distance_contrastive.mean())}")

Mean distance for triplet loss: 1.8708646297454834
Mean distance for contrastive loss: 1.609084963798523


### Create Dataset

In [ ]:
import pandas as pd

train_df = pd.read_csv(train_csv)

def create_npz_file(df):
    df_paths = df["path"].values

    df_img_arrs = []
    for path in df_paths:
        img = Image.open(path)
        img = img.resize((160, 160))
        img_arr = np.array(img).transpose(2, 0, 1) / 255
        df_img_arrs.append(img_arr)
        
    df_img_arrs = np.array(df_img_arrs)
    np.savez("lfw_cropped_train.npz", df_img_arrs)
        
create_npz_file(train_df)        

### Create a Classifier

In [ ]:
# develop a classifier for the 5 Celebrity Faces Dataset
from sklearn.preprocessing import LabelEncoder, accuracy_score, Normalizer
from sklearn.svm import SVC


# load dataset
data = load('5-celebrity-faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Dataset: train=%d, test=%d' % (trainX.shape[0], testX.shape[0]))
# normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)
# fit model
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy)
# predict
yhat_train = model.predict(trainX)
yhat_test = model.predict(testX)
# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))